In [6]:
import pandas as pd
import numpy as np
import pymongo
import dns
import datetime as dt

In [7]:
coordinates_file = '../data/coordinates.csv'
data_file = '../data/raw_data.xlsx'

In [8]:
data = pd.read_excel(data_file)
coordinates = pd.read_csv(coordinates_file)

In [4]:
username = ''
password = ''
conn = 'mongodb+srv://' + username +':' + password + '@cluster0-paegd.mongodb.net/test?retryWrites=true&w=majority'
client = pymongo.MongoClient(conn)
db = client['beer_db']


InvalidURI: The empty string is not valid username.

In [9]:
data.drop(['ServiceCenter', 'Zone', 'AccNumber', 'SystemName', 'StationName', 'TowerID', 'TowerSortOrder', 'FaucetCount', 'Distributor', 'Technician', 'DayOfService', 'TripTitle','TripCancelReason'], axis=1,inplace=True)

In [10]:
coordinates.columns.values[0] = "LocationID"

In [11]:
merge = pd.merge(data, coordinates, how='left', on='LocationID')

In [12]:
merge.dropna(inplace=True)

In [13]:
merge['Location'] = merge['Location'].str.replace('.','')
merge['BreweryName'] = merge['BreweryName'].str.replace('.','')
merge['Address'] = merge['Address'].str.replace('.','')
merge['BrandName'] = merge['BrandName'].str.replace('.','')
merge['Location'] = merge['Location'].str.replace('2M - ','')
merge['Location'] = merge['Location'].str.replace('NP - ','')
merge['Location'] = merge['Location'].str.replace('OOB - ','')
merge['Location'] = merge['Location'].str.replace('Duplicate - ','')
merge['Location'] = merge['Location'].str.replace('BO - ','')
merge['Location'] = merge['Location'].str.replace('TC - ','')
merge['Location'] = merge['Location'].str.replace('Seasonal - ','')

In [14]:
merge.head()

,LocationID,Location,Address,City,ZIP,state,BrandName,BreweryName,Active Recurring Trip,Service Location - Description,Service Location - Address Line 1,Service Location - City,Service Location - Postal Code,Service Location - State,Latitude,Longitude
178,2997,19 Handles (Sacramento/Arden),4235 Arden Way,Sacramento,95864.0,CA,Alaskan Hopothermia,Alaskan Brewing Co,1,19 Handles (Sacramento/Arden),4235 Arden Way,Sacramento,95864.0,CA,38.595658,-121.365601
179,2997,19 Handles (Sacramento/Arden),4235 Arden Way,Sacramento,95864.0,CA,Anderson Valley Boont Amber Ale,Anderson Valley,1,19 Handles (Sacramento/Arden),4235 Arden Way,Sacramento,95864.0,CA,38.595658,-121.365601
180,2997,19 Handles (Sacramento/Arden),4235 Arden Way,Sacramento,95864.0,CA,Angry Orchard,Boston Beer Co,1,19 Handles (Sacramento/Arden),4235 Arden Way,Sacramento,95864.0,CA,38.595658,-121.365601
181,2997,19 Handles (Sacramento/Arden),4235 Arden Way,Sacramento,95864.0,CA,Ballast Point Sculpin IPA,Constellation Brands,1,19 Handles (Sacramento/Arden),4235 Arden Way,Sacramento,95864.0,CA,38.595658,-121.365601
182,2997,19 Handles (Sacramento/Arden),4235 Arden Way,Sacramento,95864.0,CA,Bear Republic Thru The Haze,Bear Republic Brewing Co,1,19 Handles (Sacramento/Arden),4235 Arden Way,Sacramento,95864.0,CA,38.595658,-121.365601


In [119]:
unique_id = merge['LocationID'].unique()

In [131]:
collection = db['accounts']
date = dt.datetime.now().strftime('%x')


for x in unique_id:
    df = merge[merge['LocationID'] == x]
    obj = {
        'date': str(date),
        'id':int(df['LocationID'].iloc[0]),
        'location':str(df['Location'].iloc[0]),
        'address':str(df['Address'].iloc[0]),
        'city':str(df['City'].iloc[0]),
        'zipcode':int(df['ZIP'].iloc[0]),
        'state':str(df['state'].iloc[0]),
        'lat':float(df[' Latitude'].iloc[0]),
        'lng':float(df[' Longitude'].iloc[0]),
        'taps': []
    }
    for index, row in df.iterrows():
        taps = {row['BreweryName']: row['BrandName']}
        obj['taps'].append(taps)

    id = collection.insert_one(obj).inserted_id

In [15]:
unique_taps = merge['BrandName'].unique()

In [16]:
# collection = db['taps']
date = dt.datetime.now().strftime('%x')
for x in unique_taps:
    df = merge[merge['BrandName'] == x]
    obj = {
        'date': str(date),
        'tap':str(df['BrandName'].iloc[0]),
        'brewery':str(df['BreweryName'].iloc[0]),
        'locations': []
    }
    unique_locations = df['LocationID'].unique()
    for y in unique_locations:
        tf = df[df['LocationID'] == y]
        
        locations = {
        'id': int(tf['LocationID'].iloc[0]),
        'location':str(tf['Location'].iloc[0]),
        'address':str(tf['Address'].iloc[0]),
        'city':str(tf['City'].iloc[0]),
        'zipcode':int(tf['ZIP'].iloc[0]),
        'state':str(tf['state'].iloc[0]),
        'lat':float(tf[' Latitude'].iloc[0]),
        'lng':float(tf[' Longitude'].iloc[0]),
            
        }
        obj['locations'].append(locations)
#     id = collection.insert_one(obj).inserted_id



In [17]:
obj

{'date': '03/12/20',
 'tap': 'Revision Project Humulus Lupulus',
 'brewery': 'Revision',
 'locations': [{'id': 4921,
   'location': 'Zocalo Mexican (Sacramento/Capitol)',
   'address': '1801 Capitol Ave',
   'city': 'Sacramento',
   'zipcode': 95811,
   'state': 'CA',
   'lat': 38.57418,
   'lng': -121.4838}]}